In [3]:
# 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes==0.43.1

Found existing installation: bitsandbytes 0.48.2
Uninstalling bitsandbytes-0.48.2:
  Successfully uninstalled bitsandbytes-0.48.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.3 MB/s eta 0:00:00


In [22]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

FINETUNED = "/content/drive/MyDrive/Scheduler_project/exaone-calendar-lora"

tok = AutoTokenizer.from_pretrained(FINETUNED)

model = AutoPeftModelForCausalLM.from_pretrained(
    FINETUNED,
    device_map="auto",
    offload_folder="/content/offload",
    torch_dtype="bfloat16"
)

print("✅ AutoPeftModel로 안전하게 LoRA 불러오기 완료")



✅ AutoPeftModel로 안전하게 LoRA 불러오기 완료


In [29]:
import re, json

# Intent 분류
def classify_intent(user_text):
    messages = [
      {"role":"system","content":"You are an intent classifier. Classify the user request into 'calendar' or 'chat'. If the user's speech is a general conversation, select chat. If the conversation is related to a schedule, select calendar. Respond ONLY with one word."},
      {"role":"user","content": user_text}
    ]
    inputs = tok.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt", return_dict=True).to(model.device)
    out = model.generate(**inputs, max_new_tokens=10)
    txt = tok.decode(out[0], skip_special_tokens=True).strip().lower()
    return "calendar" if "calendar" in txt else "chat"



In [24]:
#캘린더 JSON 생성함수 -> intent가 calendar인 경우 실행
def generate_calendar_json(user_text):
    messages = [
      {"role":"system","content":"You are a calendar assistant. Always respond with ONLY valid JSON in UTF-8 matching this schema: {\"actions\":[{\"action\":\"create|read|update|delete\",\"event\":{\"title\":string|null,\"date\":string(YYYY-MM-DD)|null,\"time\":string|null,\"participants\":[string],\"location\":string|null}}]}."},
      {"role":"user","content": user_text}
    ]
    inputs = tok.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt", return_dict=True).to(model.device)
    out = model.generate(**inputs, max_new_tokens=256)
    txt = tok.decode(out[0], skip_special_tokens=True).strip()
    m = re.search(r'(\{.*\})', txt, flags=re.S)
    if not m:
        return None, txt
    try:
        return json.loads(m.group(1)), txt
    except json.JSONDecodeError:
        return None, txt


In [25]:
def assistant(user_text):
    intent = classify_intent(user_text)
    print(f"[Intent] {intent}")

    if intent == "calendar":
        data, raw = generate_calendar_json(user_text)
        print("📅 JSON 응답:", data or raw)
    else:
        # 여기엔 상담사 모드 프롬프트를 붙일 수 있음
        print("💬 Chat 응답:", "상담사 모드로 대화하기 (추후 확장)")


In [31]:
assistant("I'm too tired")

[Intent] calendar


KeyboardInterrupt: 